# List of My Movies

In [110]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-03-07 19:47:45.466716


# Combine Movies

In [160]:
%load_ext autoreload
%autoreload

from combine import combine

from wikipedia import wikipedia
from wikifilm import wikifilm
from oscar import oscars
from razzies import razzies
from rollingstone import rollingstone
from BAFTA import BAFTA
from SAG import SAG
from filmsite import filmsite
from films101 import films101
from AACTA import AACTA
from flops import flops
from canada import Canada
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from ultimatemovierankings import ultimatemovierankings

comb = combine()
comb.setOscarData(oscars(wikipedia()), 10)
comb.setWikiFilmData(wikifilm(), 1)
comb.setRazziesData(razzies(), 1)
comb.setBAFTAData(BAFTA(), 1)
comb.setSAGData(SAG(), 1)
comb.setAACTAData(AACTA(), 1)
comb.setFilmsiteData(filmsite(), 1)
comb.setFilms101Data(films101(), 1)
comb.setFlopsData(flops(), 1)
comb.setCanadaData(Canada(), 1)
comb.setRollingStoneData(rollingstone(), 1)
comb.setRottenTomatoesData(rottentomatoes(), 90)
comb.setBoxOfficeMojoData(boxofficemojo(), 5e6)
comb.setUltimateMovieRankingsData(ultimatemovierankings(), 90)
comb.getData()
comb.saveCorrections(debug=True)
comb.mergeMovies(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 23 Years of Rolling Stone Movies
Found 47 Years of BAFTA Movies
Found 38 Years of Razzies Movies
Found 84 Years of Oscar Movies
Found 94 Years of Rotten Tomatoes Movies
Found 105 Years of Filmsite Movies
Found 25 Years of SAG Movies
Found 39 Years of Box Office Mojo Movies
Found 119 Years of Films101 Movies
Found 50 Years of AACTA Movies
Found 67 Years of Ultimate Movie Rankings Movies
Found 32 Years of Flops Movies
Found 98 Years of Wiki Film Movies
Found 61 Years of Canada Movies
Found Data Between 1900 and 2018
Found 908 old corrections
Found 1 new corrections
Adding Distant Voices, Still Lives
Saving data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
Saved data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
  --> This file is 52.4kB.
There are 909 total corrections
Found 17744 movies
	 ('Films101', 8274)
	 ('Box Office Mojo', 3934)
	 ('Oscar', 2268)
	 (

# Merge My Movies With Combined Movies

In [174]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine       = None
        self.missingMovies = None
        self.status        = None
        self.renames       = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getCombinedMoviesByYear(self, debug=False): 
        combmovies = self.getCombinedMovies()
        moviesByYear = {}
        for movie in combmovies.keys():
            name = movie[:-7]
            try:
                year = int(movie[-5:-1])
            except:
                raise ValueError("Error with {0} --> {1}".format(movie, combmovies[movie]))
            if moviesByYear.get(year) is None:
                moviesByYear[year] = {}
            if moviesByYear[year].get(name) is not None:
                raise ValueError("You have a duplicate movie for {0}!".format(movie))
            moviesByYear[year][name] = combmovies[movie]
            
        moviesByYear = self.removeDuplicates(moviesByYear)
        return moviesByYear

    def removeDuplicates(self, combinedMovies):
        cmovies  = combinedMovies
        ordering = self.combine.getOrdering()
        years    = cmovies.keys()

        drops  = {}
        iDrops = 0
        while len(drops) > 0 or iDrops == 0:
            #print("Iteration: {0}".format(iDrops))
            if iDrops > 0:
                drops = {}

            for year in years:
                movies = cmovies[year]

                for name,source in movies.items():
                    key = " :: ".join([str(year),name,source])

                    if len(name.strip()) == 0:
                        if drops.get(key) is None:
                            drops[key] = []
                        drops[key].append(key)
                        continue

                    if name.endswith(" 3D"):
                        if drops.get(key) is None:
                            drops[key] = []
                        drops[key].append(key)
                        continue

                    key = " :: ".join([str(year),name,source])
                    for yearval in [year-x for x in [-2,-1,1,2]]:
                        if cmovies.get(yearval) is not None:
                            if cmovies[yearval].get(name) is not None:
                                foundsource = cmovies[yearval][name]
                                fKey = " :: ".join([str(yearval),name,foundsource])


                                #print(year,'\t',name,'\tFound from {0} in {1} from {2}'.format(source,yearval,foundsource), end="\t\t")

                                iSource  = ordering.index(source)
                                ifSource = ordering.index(foundsource)                    

                                if iSource == ifSource:
                                    if yearval > year:
                                        #print("\t====> Keeping",end="")
                                        if drops.get(key) is None:
                                            drops[key] = []
                                        drops[key].append(fKey)



                                if iSource < ifSource:
                                    #print("\t====> Keeping",end="")
                                    if drops.get(key) is None:
                                        drops[key] = []
                                    drops[key].append(fKey)

                                #print("")

            for key,drop in drops.items():
                for fKey in drop:
                    year,name,source = fKey.split(" :: ")
                    try:
                        del cmovies[int(year)][name]
                    except:
                        pass
                        #print("Could not delete {0}/{1}".format(year,name))

            iDrops += 1

        return cmovies

    
    def printMissingMoviesByYear(self, year, movies):
        print("")
        for movie in sorted(movies):
            print("{0: <10}{1: <100}".format(year, movie))


    def printMissingMovies(self, year=None):
        missingMovies = self.getMissingMovies()
        print("====================== Missing Movies ======================")
        print("{0: <10}{1: <100}".format("Year", "Movie"))
        print("{0: <10}{1: <100}".format("----", "-----"))

        if year is None:
            for year in missingMovies.keys():
                self.printMissingMoviesByYear(missingMovies[year])
        else:        
            self.printMissingMoviesByYear(year, missingMovies[year])    


    
    def setMissingMovies(self, missing):
        self.missingMovies = missing
        
    def getMissingMovies(self):
        return self.missingMovies

    
    def setCombinedMovieStatus(self, status):
        self.status = status
        
    def getCombinedMovieStatus(self):
        return self.status

    
    def setMovieRenames(self, renames):
        self.renames = renames
        
    def getMovieRenames(self):
        return self.renames
        
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueError("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
        
    def getMyMoviesByYear(self, debug=False): 
        mymovies = self.getMyMovies()
        moviesByYear = {}
        for movie in mymovies.keys():
            name = movie[:-7]
            try:
                year = int(movie[-5:-1])
            except:
                raise ValueError("Error with {0} --> {1}".format(movie, mymovies[movie]))
            if moviesByYear.get(year) is None:
                moviesByYear[year] = {}
            if moviesByYear[year].get(name) is not None:
                raise ValueError("You have a duplicate movie for {0}!".format(movie))
            moviesByYear[year][name] = mymovies[movie]
            
        return moviesByYear
            
            
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*.*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8, num=3):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), num, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            location = moviesMine[result[0]]
            print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), num, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                location = moviesMine[result[0]]
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            vs=["of", "and", "the", "a"]
            for v in vs:
                name = name.replace(" {0} ".format(v), " {0} ".format(v.title()))
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
            
            
    
    def showCombinedMovieStatus(self, year=None):
        locations = self.getCombinedMovieStatus()
        years = sorted(locations.keys())
        if year is not None:
            years = [year]

        print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))


        for year in years:
            print('\n',129*'-')
            for name,sourcedata in locations[year].items():
                location = sourcedata[0]
                source   = sourcedata[1]
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, name, source))

        
        
    def mergeMovies(self, toFile=False, movie=None, source=None, year=None):
        combinedMovies = self.getCombinedMoviesByYear()
        
        myMovies       = self.getMyMoviesByYear()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")
        
        
        yearcut = year
        sourcecut = source
        moviecut = movie

        pYear = None
        yearRenames = {}
        
        locations = {}
        toGet = {}
        

        years = sorted(combinedMovies.keys())
        for year in years:
            locations[year] = {}
            movies = combinedMovies[year]
            #print("Analyzing {0} movies from {1}".format(len(movies), year))

            for name,source in movies.items():
                key = "{0} [{1}]".format(name, year)
                locations[year][name] = [None, source]
                
                for yearval in [year+y for y in [0,1,-1,2,-2]]:
                    try:
                        location = myMovies[yearval][name]
                        break
                    except:
                        location = None

                exact    = False
                if location is not None:
                    location = u'\u2713'
                    exact    = True
                else:
                    nearbyMovies = []
                    for yearval in [year+y for y in [0,1,-1]]:
                        yearlyMovies = myMovies.get(yearval)                        
                        if isinstance(yearlyMovies, dict):
                            result = findNearest(name, yearlyMovies.keys(), 1, 0.98)
                            if len(result) > 0:
                                location = u'\u2248'
                                
                                myMovie = myMovies[yearval][result[0]]
                                fileInfo = getFileBasics(myMovie)
                                newMovieKey = "{0} [{1}]".format(name, year)
                                yearRenames[myMovie] = join(fileInfo[0], "".join([newMovieKey, fileInfo[2]]))

                    if location is None:
                        for yearval in [year+y for y in [0,1,-1]]:
                            yearlyMovies = myMovies.get(yearval)
                            if isinstance(yearlyMovies, dict):
                                result = findNearest(name, yearlyMovies.keys(), 1, 0.90)
                                if len(result) > 0:
                                    location = u'\u2047'

                                    myMovie = myMovies[yearval][result[0]]
                                    fileInfo = getFileBasics(myMovie)
                                    newMovieKey = "{0} [{1}]".format(name, year)
                                    yearRenames[myMovie] = join(fileInfo[0], "".join([newMovieKey, fileInfo[2]]))

                    if location is None:
                        location = ""
                        
                    if exact is False:
                        if toGet.get(year) is None:
                            toGet[year] = {}
                        toGet[year][name] = source
                
                locations[year][name][0] = location
         
        
        self.setCombinedMovieStatus(locations)
        self.setMissingMovies(toGet)
        self.setMovieRenames(yearRenames)

In [175]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Thu Mar 07, 2019 21:49:49 for Last Run


In [153]:
mine.suggestRenames()

In [154]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5738 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 464.9kB.
Current Time is Thu Mar 07, 2019 21:11:51 for Last Run


In [155]:
mine.searchMyMovies("Cabin Boy", 1994, cutoff=0.6)

Cabin Boy  --->  ['Casino [1995]', 'Bad Boys [1995]', 'Chasing Amy [1997]']
moveFile(src="/Volumes/Seagate/Movies/Casino [1995].mp4", dst="/Volumes/Seagate/Movies/Casino [1995].mp4", debug=True)



In [103]:
#mine.getMyMoviesByYear()[1900]

In [176]:
### Stopped after 1985. Do 1986 onward
mine.mergeMovies()

In [171]:
show = False
if show:
    renames = mine.getMovieRenames()
    for old,new in renames.items():
        print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)".format(old,new))

In [157]:

moveFile(src="/Volumes/Download/Movies/The Navigator A Medieval Odyssey [1988].mp4", dst="/Volumes/Download/Movies/The Navigator: A Medieval Odyssey [1988].mp4", debug=True)

Moving /Volumes/Download/Movies/The Navigator A Medieval Odyssey [1988].mp4 to /Volumes/Download/Movies/The Navigator: A Medieval Odyssey [1988].mp4


In [183]:
#mine.showCombinedMovieStatus(1987)
mine.printMissingMovies(2000)

====================== Missing Movies ======================
Year      Movie                                                                                               
----      -----                                                                                               

2000      101 Reykjavík                                                                                       
2000      15 Amore                                                                                            
2000      3 Strikes                                                                                           
2000      A Time For Drunken Horses                                                                           
2000      Aimée & Jaguar                                                                                      
2000      Ali Zaoua, prince de la rue                                                                         
2000      All The Pretty Horses                   

In [173]:
mine.missingMovies

{1900: {None},
 1901: {None},
 1902: {None},
 1903: {None},
 1904: {None},
 1905: {None},
 1906: {None},
 1907: {None},
 1908: {None},
 1909: {None},
 1910: {None},
 1911: {None},
 1912: {None},
 1913: {None},
 1914: {None},
 1915: {None},
 1916: {None},
 1917: {None},
 1918: {None},
 1919: {None},
 1920: {None},
 1921: {None},
 1922: {None},
 1923: {None},
 1924: {None},
 1925: {None},
 1926: {None},
 1927: {None},
 1928: {None},
 1929: {None},
 1930: {None},
 1931: {None},
 1932: {None},
 1933: {None},
 1934: {None},
 1935: {None},
 1936: {None},
 1937: {None},
 1938: {None},
 1939: {None},
 1940: {None},
 1941: {None},
 1942: {None},
 1943: {None},
 1944: {None},
 1945: {None},
 1946: {None},
 1947: {None},
 1948: {None},
 1949: {None},
 1950: {None},
 1951: {None},
 1952: {None},
 1953: {None},
 1954: {None},
 1955: {None},
 1956: {None},
 1957: {None},
 1958: {None},
 1959: {None},
 1960: {None},
 1961: {None},
 1962: {None},
 1963: {None},
 1964: {None},
 1965: {None},
 1966: {No

In [166]:
printMissingMovies(1995)

====================== Missing Movies ======================
Year      Movie                                                                                               
----      -----                                                                                               

{None}


TypeError: unsupported format string passed to NoneType.__format__

In [51]:

moveFile(src="/Volumes/Save/Movies/Sweeney Todd The Demon Barber Of Fleet Street [2007].mp4", dst="/Volumes/Save/Movies/Sweeney Todd: The Demon Barber Of Fleet Street [2007].mp4", debug=True)

moveFile(src="/Volumes/Download/Movies/The King Of Kong A Fistful Of Quarters [2007].mp4", dst="/Volumes/Download/Movies/The King of Kong: A Fistful of Quarters [2007].mp4", debug=True)

Moving /Volumes/Save/Movies/Sweeney Todd The Demon Barber Of Fleet Street [2007].mp4 to /Volumes/Save/Movies/Sweeney Todd: The Demon Barber Of Fleet Street [2007].mp4
Moving /Volumes/Download/Movies/The King Of Kong A Fistful Of Quarters [2007].mp4 to /Volumes/Download/Movies/The King of Kong: A Fistful of Quarters [2007].mp4


In [40]:

moveFile(src="/Volumes/Seagate/Movies/X The Man With The X Ray Eyes [1963].mp4", dst="/Volumes/Seagate/Movies/X: The Man With The X-Ray Eyes [1963].mp4", debug=True)

Moving /Volumes/Seagate/Movies/X The Man With The X Ray Eyes [1963].mp4 to /Volumes/Seagate/Movies/X: The Man With The X-Ray Eyes [1963].mp4


In [33]:
mine.searchMyMovies("Goldeneye", 1995, cutoff=0.6, num=10)

Goldeneye  --->  ['James Bond Goldeneye [1995]', 'Powder [1995]', 'Volunteers [1985]', 'Jade [1995]', '12 Monkeys [1995]', 'Sleepers [1996]', 'Clueless [1995]', 'Wilde [1997]', 'Se7en [1995]', 'Nixon [1995]']
moveFile(src="/Volumes/Seagate/Movies/James Bond Goldeneye [1995].mkv", dst="/Volumes/Seagate/Movies/James Bond Goldeneye [1995].mkv", debug=True)

